In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('SBA 345').getOrCreate()
stock = spark.read.load("CompanyABC_stock.csv", format="csv", header = True,inferSchema = True)

In [4]:
stock.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [5]:
stock.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [8]:
stock.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [11]:
stock.show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000000004|
|2012-01-10|             59.43|59.709998999999996|             5

In [12]:
stock.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [3]:
stock = stock.withColumn('HV Ratio', stock['Volume'] / stock['High'])
stock.show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|          HV Ratio|
+----------+------------------+------------------+------------------+------------------+--------+------------------+------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|207481.16266817617|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475| 158961.0657485026|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|214159.68155249383|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|135734.22816258657|
|2012-01-09|         59.029999|         59.54999

In [14]:
stock.orderBy(stock.High.desc()).show(5)

+----------+-----------------+-----------------+---------+---------+--------+-----------------+
|      Date|             Open|             High|      Low|    Close|  Volume|        Adj Close|
+----------+-----------------+-----------------+---------+---------+--------+-----------------+
|2015-01-13|        90.800003|        90.970001|    88.93|89.309998| 8215400|        83.825448|
|2015-01-08|        89.209999|90.66999799999999|    89.07|90.470001|12713600|84.91421600000001|
|2015-01-09|            90.32|        90.389999|    89.25|89.349998| 8522500|        83.862993|
|2015-01-12|        89.360001|        90.309998|89.220001|90.019997| 7372500|        84.491846|
|2015-01-23|88.41999799999999|        89.260002|87.889999|88.510002| 7565800|83.07458100000001|
+----------+-----------------+-----------------+---------+---------+--------+-----------------+
only showing top 5 rows



In [18]:
from pyspark.sql.functions import mean,avg,max,min

stock.select(mean("Close")).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [19]:
stock.select(max("Volume")).show()
stock.select(min("Volume")).show()

+-----------+
|max(Volume)|
+-----------+
|   80898100|
+-----------+

+-----------+
|min(Volume)|
+-----------+
|    2094900|
+-----------+



In [33]:
stock.createOrReplaceTempView("stock")
spark.sql("SELECT count(Close) FROM stock WHERE Close < 70").show()
#also this one below
stock.select('Close').filter(stock.Close < 70).count()

+------------+
|count(Close)|
+------------+
|         397|
+------------+



397

In [42]:
stock.select('High').filter(stock.High > 80).count()/stock.select('Date').count() * 100

9.141494435612083

In [4]:
stock.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/classicmodels") \
  .option("dbtable", "companyabc_db.stockdata") \
  .option("user", "root") \
  .option("password", "password") \
  .save()

# Section Two

In [46]:
sales = spark.read.load(['Sales_February_2019.csv','Sales_April_2019.csv'], format="csv", header = True,inferSchema = True)

In [51]:
sales.columns

['Order ID',
 'Product',
 'Quantity Ordered',
 'Price Each',
 'Order Date',
 'Purchase Address']

In [52]:
sales.printSchema()

root
 |-- Order ID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: integer (nullable = true)
 |-- Price Each: double (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)



In [53]:
sales.describe().show()

+-------+------------------+------------+-------------------+------------------+--------------+--------------------+
|summary|          Order ID|     Product|   Quantity Ordered|        Price Each|    Order Date|    Purchase Address|
+-------+------------------+------------+-------------------+------------------+--------------+--------------------+
|  count|             30275|       30328|              30275|             30275|         30328|               30328|
|   mean|173816.63355904212|        null| 1.1239966969446737| 183.7621383980421|          null|                null|
| stddev|14902.179833784816|        null|0.43431295924749175|328.79347229907665|          null|                null|
|    min|            150502|20in Monitor|                  1|              2.99|02/01/19 01:51|1 14th St, New Yo...|
|    max|            194094|      iPhone|                  7|            1700.0|    Order Date|    Purchase Address|
+-------+------------------+------------+-------------------+---

In [55]:
sales.show()

+--------+--------------------+----------------+----------+---------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|     Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+---------------+--------------------+
|  176558|USB-C Charging Cable|               2|     11.95| 4/19/2019 8:46|917 1st St, Dalla...|
|  176559|Bose SoundSport H...|               1|     99.99| 4/7/2019 22:30|682 Chestnut St, ...|
|  176560|        Google Phone|               1|     600.0|4/12/2019 14:38|669 Spruce St, Lo...|
|  176560|    Wired Headphones|               1|     11.99|4/12/2019 14:38|669 Spruce St, Lo...|
|  176561|    Wired Headphones|               1|     11.99| 4/30/2019 9:27|333 8th St, Los A...|
|  176562|USB-C Charging Cable|               1|     11.95|4/29/2019 13:03|381 Wilson St, Sa...|
|  176563|Bose SoundSport H...|               1|     99.99|  4/2/2019 7:46|668 Center St, Se...|
|  176564|USB-C Charging Cable

In [60]:
sales = sales.withColumn('Total Price', sales['Price Each'] * sales['Quantity Ordered'])
sales = sales.select("Order ID", "Total Price", "Price Each", "Quantity Ordered", "Product", "Order Date", "Purchase Address")
sales.show()

+--------+-----------+----------+----------------+--------------------+---------------+--------------------+
|Order ID|Total Price|Price Each|Quantity Ordered|             Product|     Order Date|    Purchase Address|
+--------+-----------+----------+----------------+--------------------+---------------+--------------------+
|  176558|       23.9|     11.95|               2|USB-C Charging Cable| 4/19/2019 8:46|917 1st St, Dalla...|
|  176559|      99.99|     99.99|               1|Bose SoundSport H...| 4/7/2019 22:30|682 Chestnut St, ...|
|  176560|      600.0|     600.0|               1|        Google Phone|4/12/2019 14:38|669 Spruce St, Lo...|
|  176560|      11.99|     11.99|               1|    Wired Headphones|4/12/2019 14:38|669 Spruce St, Lo...|
|  176561|      11.99|     11.99|               1|    Wired Headphones| 4/30/2019 9:27|333 8th St, Los A...|
|  176562|      11.95|     11.95|               1|USB-C Charging Cable|4/29/2019 13:03|381 Wilson St, Sa...|
|  176563|      99.

In [62]:
sales.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/classicmodels") \
  .option("dbtable", "companyabc_db.sales") \
  .option("user", "root") \
  .option("password", "password") \
  .save()